# Data labeling

#### Requirements

In [ ]:
!pip install pandas
!pip install google-cloud-bigquery
!pip install google-cloud-storage
!pip install pandas-gbq
!pip install Flask

Este notebook está dedicado a etiquetar los datos que hemos recogido anteriormente.
Para ellos haremos uso del micro-servidor we Flask.


## Usage

Para empezar a usar el etiquetador lo primero que tenemos que hacer es ejecutar todas las siguientes celdas de este Notebook.

Cuanto ejecutemos la última celda debtenemos que abrir una nueva pestaña en el navegador (Google Chrome) y poner en ella la dirección http que muestra el mensaje al ejecutar la celda.

Ahora podemos empezar a etiquetar.

En la página veremos un formulario a la izquierda y una imagen (pantallazo de una página web) a la derecha.

Queremos etiquetar, si los hay, los siguientes elementos "mensaje de cookies", "botón de aceptar cookies" y "botón de cerrar cookies". Si no hay mensaje de cookies le damos al boton next. Si está el mensaje pero no alguno de los botones no pasa nada. Para ello seguiremos los siguiente pasos:


1. Seleccionar el tipo de página
2. Pinchamos en el checkbox "Texto Cookies", localizamos el elemento de la página "mensaje de cookies", pinchamos con el ratón encima del texto del elemento y en ese momento en el input "Texto Cookies" deberían aparecer unas coordenadas x,y y deseleccionarse el checkbox.
3. Repetimos el paso 2 con los elemento aceptar cookies y cerrar cookies, si los hay, respectivamente.
4. Pinchamos en el botón save and next.

## Functions

In [ ]:
def get_next_url_and_screenshot_scraped():
    import os
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'tfmfabri-2cf8db9433ba.json'

    from google.cloud import bigquery
    client = bigquery.Client()
    
    query = 'SELECT distinct url,created_date,ts FROM `tfmfabri.tfm.tfm_urls` where labeled != true and scraped = true order by created_date desc limit 1'
    query_job = client.query(query)
    results = query_job.result()
    for row in results:
        url = row['url']
        ts = str(row['ts'])

    from google.cloud import storage
    client_storae = storage.Client()
    # https://console.cloud.google.com/storage/browser/[bucket-id]/
    bucket = client_storae.get_bucket('tfm_bucket')
    
    screenshot_url = 'https://tfm_bucket.storage.googleapis.com/'+ts+'_screenshot.png'

    return url, screenshot_url




def update_urls_gbq_empty(url):
    import os
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'tfmfabri-2cf8db9433ba.json'

    from google.cloud import bigquery
    client = bigquery.Client()
    query = "UPDATE `tfm.tfm_urls` set labeled = true where url = '"+url+"'"
    query_job = client.query(query)
    results = query_job.result()
    return "ok"
    
def update_urls_gbq_labeled(label,url):
    import os
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'tfmfabri-2cf8db9433ba.json'

    from google.cloud import bigquery
    client = bigquery.Client()
    import datetime
    datenow = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
    labeled_date = datetime.datetime.strptime(datenow,'%Y-%m-%d %H:%M')

    query = "UPDATE `tfm.tfm_urls` set labeled = true, labeled_date = '"+str(labeled_date)+"' ,label = '"+label+"'  where url = '"+url+"'"


        
    query_job = client.query(query)
    results = query_job.result()
    return "ok"

def update_label_cookies_gbq(url,cookies,aceptar,cerrar,precio,titulo):
    import os
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'tfmfabri-2cf8db9433ba.json'

    from google.cloud import bigquery
    client = bigquery.Client()
    import datetime
    datenow = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
    labeled_date = datetime.datetime.strptime(datenow,'%Y-%m-%d %H:%M')

    query = "insert `tfmfabri.tfm.tfm_label_cookies`(url,cookies,aceptar,cerrar,precio,titulo) values ('"+url+"','"+cookies+"','"+aceptar+"','"+cerrar+"','"+precio+"','"+titulo+"')"
    print("query")
    print(query)

        
    query_job = client.query(query)
    results = query_job.result()
    return "ok"



In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'tfmfabri-2cf8db9433ba.json'
from google.cloud import storage
client = storage.Client()
# https://console.cloud.google.com/storage/browser/[bucket-id]/
bucket = client.get_bucket('tfm_bucket')

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'tfmfabri-2cf8db9433ba.json'

from google.cloud import bigquery
client = bigquery.Client()

## Flask App

In [ ]:
html = '''
<html>
<head>

</head>

<body>



<div id="div-data-form">
<form id="data-form">
	<div>
		<input id="button-next" type="button" value="Next"/>
		<input id="button-next-and-save" type="button" value="Save and Next"/>
	</div>
    
    <div>
        <p id="web-url"></p>
    </div>
    
	<label for="div-cookies">Tipo de página</label>
	<select name="page-type">

		<option>product</option>
		<option>category</option>
        <option>post</option>
        <option>foro</option>
		<option>home</option>		
		<option>other</option>
        <option>error</option>
        <option>popup</option>

</select>

	</br>
	</br>
	<label for="div-cookies">Texto Cookies</label>
	<input type="checkbox" name="checkbox-cookies" i/>
  	<input type="text" name="cookies" id="div-cookies" value=""><br>
	</br>
	</br>
	<label for="div-cookies">Aceptar Cookies</label>
	<input type="checkbox" name="checkbox-aceptar" i/>
  	<input type="text" name="aceptar" id="div-aceptar" value=""><br>	</br>
	</br>
	<label for="div-cookies">Cerrar Cookies</label>
	<input type="checkbox" name="checkbox-cerrar" i/>
  	<input type="text" name="cerrar" id="div-cerrar" value=""><br>	</br>
	</br>
    <!--
	<label for="div-cookies">Precio</label>
	<input type="checkbox" name="checkbox-precio" i/>
  	<input type="text" name="precio" id="div-precio" value=""><br>	</br>
	</br>
	<label for="div-cookies">Título</label>
	<input type="checkbox" name="checkbox-titulo" i/>
  	<input type="text" name="titulo" id="div-titulo" value=""><br>
    -->
</form>
</div>
<div id="div-image">
	<img id="img-to-label" src="https://storage.googleapis.com/tfm_bucket/url-labeler/loading.gif" />
</div>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.11.3/jquery.min.js"></script>  
<script src="./main.js"></script>
<style>
	#div-image{
		display:inline-block;
		/*width: 70%;*/
		overflow: auto; /* adds scrollbars */
        margin-left: 305px;

	}


	#div-image > img{
		/*max-width: 1000px;*/
	}
	#div-data-form{
		position:fixed;
        width: 300px;
	    /*width: 30%;*/
    	display: inline-block;
    	float: left;
    	text-align: center;
        background-color: white;
	}
    body{
    width: 300%;
    line-height: 30px;
    font-size: 19px;
    }
    
</sytle>



</body>

</html>
'''


js = '''
$(document).ready(function() {

    function getNextImage() {
        $.ajax({
                method: "GET",
                url: "./get-next-image",
            })
            .done(function(msg) {
                var obj = JSON.parse(msg);
                var webUrl = obj[0];
                var imgSrc = obj[1];


                $("#web-url").text(webUrl);
                $("#img-to-label").attr("src", imgSrc);
            });

        return this;
    };

    function saveForm() {
        $.ajax({
                method: "GET",
                url: "./save-form",
            })
            .done(function(msg) {
                var obj = JSON.parse(msg);
                var webUrl = obj[0];
                var imgSrc = obj[1];


                $("#web-url").text(webUrl);
                $("#img-to-label").attr("src", imgSrc);
            });

        return this;
    };


    getNextImage();





    $("#button-next").on("click", function(event) {
        $('#data-form').trigger("reset");

        $("#img-to-label").attr("src", "https://storage.googleapis.com/tfm_bucket/url-labeler/loading.gif");


        var webUrl = $('#web-url').text();
        $.ajax({
                method: "POST",
                url: "./save-next",
                data: JSON.stringify({
                    "url": webUrl
                }),
                contentType: "application/json; charset=utf-8",
                dataType: "json",
            })
            .always(function(msg) {
                getNextImage();
            });



    });


    $("#button-next-and-save").on("click", function(event) {
        var serializeForm = $('#data-form').serializeArray()
        $('#data-form').trigger("reset");

        $("#img-to-label").attr("src", "https://storage.googleapis.com/tfm_bucket/url-labeler/loading.gif");


        var webUrl = $('#web-url').text();
        $.ajax({
                method: "POST",
                url: "./save-form",
                data: JSON.stringify({
                    "url": webUrl,
                    "form": serializeForm
                }),
                contentType: "application/json; charset=utf-8",
                dataType: "json",
            })
            .always(function(msg) {
                getNextImage();
            });
            
            
    });


    $("#img-to-label").on("click", function(event) {
        var x = event.pageX - this.offsetLeft;
        var y = event.pageY - this.offsetTop;
    
        var xc = event.clientX - this.offsetLeft;
        var yc = event.clientY - this.offsetTop;
        console.log("xc: "+xc);
        console.log("yc: "+yc);
         var posX = $(this).position().left,
            posY = $(this).position().top;
            
        console.log("left: "+posX)
        console.log("top: "+posY);
        
        $(":checked").each(function() {
            var c_name = $(this).attr('name');
            if(c_name != undefined){
                var c_name_re = c_name.replace("checkbox-","");
                $("input[type='text'][name='" + c_name_re + "']").each(function() {
                    $(this).val(x + "," + y);
                });
            };
            
            $(this).prop("checked",false);
        });
        
        
    });



});


'''

In [ ]:
from flask import Flask
from flask import request

import json
import pandas as pd




app = Flask("My first web service")
@app.route('/hello',methods=['GET'])
def get_hello():
    
    
    return html
@app.route('/hello2',methods=['GET'])
def get_hello2():
    return "<h1>Hello 2</h1>"
@app.route('/ret_number/<int:n>',methods=['GET'])
def get_num(n):
    try:
        numb=int(n)
        output_file = ! head -n {numb} top_airports.csv
        #return !top_airports.csv head n
        #file.close()
        return json.dumps(output_file)
    except:
        return "could not find a number"
@app.route('/ret_number_2/<int:n>',methods=['GET'])
def get_num_2(n):
    numb=int(n)
    return fi.head(numb).to_json()



import json
import pandas as pd

app = Flask("My first web service")
@app.route('/',methods=['GET'])
def get_hello():
    
    
    return html

@app.route('/main.js',methods=['GET'])
def get_js():
    
    return js


@app.route('/get-next-image',methods=['GET'])
def get_next_image():
    
    next_url = get_next_url_and_screenshot_scraped()
       
    return json.dumps(next_url)



@app.route('/save-next',methods=['POST'])
def save_next():
    request.environ['CONTENT_TYPE'] = 'application/something_Flask_ignores'

    data = json.loads(request.data)
    url = data['url']
    
    update_urls_gbq_empty(url)
    
    
    return "ok"


@app.route('/save-form',methods=['POST'])
def save_form():
    request.environ['CONTENT_TYPE'] = 'application/something_Flask_ignores'

    data = json.loads(request.data)
    url = data['url']
    data_form = data['form']
    print(data_form)
    
    form_page_type = ''
    form_cookies = ''
    form_aceptar = ''
    form_cerrar = ''
    form_precio = ''
    form_titulo = ''
    
    for n in data_form:
        if n['name'] == 'page-type':
            form_page_type = n['value']

        elif n['name'] == 'cookies':
            form_cookies = n['value']
    
        elif n['name'] == 'aceptar':
            form_aceptar = n['value']

        elif n['name'] == 'cerrar':
            form_cerrar = n['value']

        elif n['name'] == 'precio':
            form_precio = n['value']

        elif n['name'] == 'titulo':
            form_titulo = n['value']
            

    update_urls_gbq_labeled(form_page_type,url)
    update_label_cookies_gbq(url,form_cookies,form_aceptar,form_cerrar,form_precio,form_titulo)
    
    
    return "ok"






@app.route('/hello2',methods=['GET'])
def get_hello2():
    return "<h1>Hello 2</h1>"
@app.route('/ret_number/<int:n>',methods=['GET'])
def get_num(n):
    try:
        numb=int(n)
        output_file = ! head -n {numb} top_airports.csv
        #return !top_airports.csv head n
        #file.close()
        return json.dumps(output_file)
    except:
        return "could not find a number"
@app.route('/ret_number_2/<int:n>',methods=['GET'])
def get_num_2(n):
    numb=int(n)
    return fi.head(numb).to_json()



In [ ]:
app.run()